In [13]:
##### This is the same as the enriched model version

# IMPORTS
 
#  Import libraries for numpy, random generator, and 
import numpy as np
import random
from collections import defaultdict, Counter

# Set the seed to 42
np.random.seed(42)
random.seed(42)

### HELPER: Proportion Weights (per disruptor group)

# Make a function that gets the proportion of mobilized, assisted and unprepared
# that come to our queuing system based on the data collected
def get_weights():
    probs = {
        "mobilized": (0.106383 + 0.037736 + 0.147727) / 3,
        "assisted":  (0.446809 + 0.132075 + 0.272727) / 3,
        "unprepared":(0.446809 + 0.188679 + 0.204545) / 3,
        }
# The proportion of normal
# people should then be 1 minus each disruptor weighed proportion
    total = sum(probs.values())
    probs["normal"] = 1 - total
    return list(probs.keys()), list(probs.values())

# Print each proportion for each group
def print_weights():
    groups, weights = get_weights()
    for group, weight in zip(groups, weights):
        print(f"{group}: {weight:.4f}")

# Use the function to print the weights
# print_weights()

### Helper: Compute Lamda (per shift)

# Compute the lambda value for the queuing system based on the total number of customers
# for each shift which is 249, 322, 446 for 8am, 12pm, and 4pm respectively
# It should be computed as (total number of people) / (shift duration in seconds)*(2 lanes)
def compute_lambda(total_customers):
    shift_duration_seconds = 30 * 60  # Shift duration in seconds (30 minutes)
    lanes = 2  # Number of lanes
    lamda = total_customers / (shift_duration_seconds * lanes)
    return lamda

# Print each lambda value for each shift to 4 decimal places
def print_lambda(total_customers):
    lamda = compute_lambda(total_customers)
    print(f"Lambda: {lamda:.4f}")

# Use the function to print the lambda value for 8am
# print_lambda(249)

### HELPER: Mixture Distribution Service Time (per disruptor group)

# Create a function that computes the service time for each disruptor group and
# a normal person
# It should use exponential distribution for disruptor groups with means of 7, 20, 21
# seconds for mobbilized, assisted, unprepared respectively 
# It should use uniform distribution for normal people between 1 and 2 seconds
def compute_service_time(group):
    if group == "mobilized":
        return np.random.exponential(7)
    elif group == "assisted":
        return np.random.exponential(20)
    elif group == "unprepared":
        return np.random.exponential(21)
    elif group == "normal":
        return np.random.uniform(1, 2)
    else:
        raise ValueError("Invalid group")
    
### HELPER: Shift Data (per shift)

# Create a function that stores lambda, lane probabilities, and number 
# of wrong exitors for each shift
# The lambda value should be computed using the compute_lambda function
# The lane probabilities should be extracted from the data and that is
    # 8am - 0.614, 0.386 for line 1 and line 2 respectively
    # 12pm - 0.632, 0.368 for line 1 and line 2 respectively
    # 4pm - 0.240, 0.760 for line 1 and line 2 respectively

def shift_data(shift_time):
    # Compute lambda for the shift
    if shift_time == "8am":
        lamda = compute_lambda(249)
    elif shift_time == "12pm":
        lamda = compute_lambda(322)
    elif shift_time == "4pm":
        lamda = compute_lambda(446)
    else:
        raise ValueError("Invalid shift time")

    # Save wrong exitors for each shift, it is 4 for 8am and 12 for
    # for the other shifts
    if shift_time == "8am":
        wrong_exitors = 4
    elif shift_time == "12pm":
        wrong_exitors = 12
    elif shift_time == "4pm":
        wrong_exitors = 12
    else:
        raise ValueError("Invalid shift time")

    # Define lane probabilities based on the shift time
    if shift_time == "8am":
        lane_probabilities = [0.614, 0.386]
    elif shift_time == "12pm":
        lane_probabilities = [0.632, 0.368]
    elif shift_time == "4pm":
        lane_probabilities = [0.240, 0.760]
    else:
        raise ValueError("Invalid shift time")
    
    # Return a dictionary with the computed values
    return {
        "lambda": lamda,
        "lane_probabilities": lane_probabilities,
        "wrong_exitors": wrong_exitors
    }



In [14]:
##### This is different than the enriched model version because it is our new model
##### based on 4 lanes instead of 2

np.random.seed(42)
random.seed(42)

def optimized_simulation_model(shift_time, num_customers=500):
    lam = shift_data(shift_time)["lambda"]
    groups, weights = get_weights()

    # Define global scanner pool: lane 1 (1 scanner), lane 2 (2), lane 3 (2), lane 4 (1)
    scanner_config = {
        1: [0],       
        2: [0, 0],    
        3: [0, 0],    
        4: [0]        
    }

    arrivals = np.zeros(num_customers)
    service_times = np.zeros(num_customers)
    wait_times = np.zeros(num_customers)
    lanes = np.zeros(num_customers, dtype=int)
    groups_array = np.zeros(num_customers, dtype=object)

    now = 0.0

    for i in range(num_customers):
        # Arrival
        interarrival_time = np.random.exponential(1 / (lam * 2))  
        now += interarrival_time

        group = np.random.choice(groups, p=weights)
        service_time = compute_service_time(group)

        # Find earliest available scanner across all lanes
        next_lane, next_scanner, free_time = min(
            ((lane, idx, t)
             for lane, scanners in scanner_config.items()
             for idx, t in enumerate(scanners)),
            key=lambda x: x[2]
        )

        # Assign wait and start
        if free_time <= now:
            wait_time = 0.0
            start = now
        else:
            wait_time = free_time - now
            start = free_time

        # Update that scanner's availability
        scanner_config[next_lane][next_scanner] = start + service_time

        # Log data
        arrivals[i] = now
        service_times[i] = service_time
        wait_times[i] = wait_time
        lanes[i] = next_lane
        groups_array[i] = group

    return {
        "arrivals": arrivals,
        "service_times": service_times,
        "wait_times": wait_times,
        "lanes": lanes,
        "groups": groups_array
    }

In [15]:
# Compute the same average wait times

np.random.seed(42)
random.seed(42)


def compute_average_waits(results):
    import pandas as pd

    df = pd.DataFrame({
        "wait_time": results["wait_times"],
        "group": results["groups"],
        "lane": results["lanes"]
    })

    avg_by_group = df.groupby("group")["wait_time"].mean().to_dict()
    avg_by_lane = df.groupby("lane")["wait_time"].mean().to_dict()
    overall_avg = df["wait_time"].mean()

    return avg_by_group, avg_by_lane, overall_avg

# Run the simulation for all shifts
for shift in ["8am", "12pm", "4pm"]:
    results = optimized_simulation_model(shift)
    avg_group, avg_lane, avg_overall = compute_average_waits(results)

    print(f"\n--- {shift} SHIFT ---")
    print("Average wait by group:", {k: f"{v:.2f}" for k, v in avg_group.items()})
    print("Average wait by lane:", {k: f"{v:.2f}" for k, v in avg_lane.items()})
    print("Overall average wait:", f"{avg_overall:.2f} seconds")



--- 8am SHIFT ---
Average wait by group: {'assisted': '0.05', 'mobilized': '0.00', 'normal': '0.00', 'unprepared': '0.00'}
Average wait by lane: {1: '0.00', 2: '0.01', 3: '0.04', 4: '0.00'}
Overall average wait: 0.02 seconds

--- 12pm SHIFT ---
Average wait by group: {'assisted': '0.17', 'mobilized': '0.00', 'normal': '0.05', 'unprepared': '0.07'}
Average wait by lane: {1: '0.06', 2: '0.13', 3: '0.07', 4: '0.03'}
Overall average wait: 0.08 seconds

--- 4pm SHIFT ---
Average wait by group: {'assisted': '0.59', 'mobilized': '0.33', 'normal': '0.29', 'unprepared': '0.29'}
Average wait by lane: {1: '0.29', 2: '0.42', 3: '0.48', 4: '0.17'}
Overall average wait: 0.38 seconds
